<a href="https://colab.research.google.com/github/vivekkishore/machinelearning/blob/main/querygpt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-dotenv
!pip install openai
!pip install --upgrade langchain
!pip install tiktoken
!pip install chromadb
!pip install unstructured
import warnings
warnings.filterwarnings('ignore')
import langchain
langchain.debug=False
import os, openai
from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv())
openai.api_key=os.environ['API_KEY']

In [2]:
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredExcelLoader
#Load
loader = UnstructuredExcelLoader("/content/Sample2.xlsx")
docs = loader.load()

# Split
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0,separators=["\n\n\n"])
texts=text_splitter.split_documents(docs)


In [3]:
texts

[Document(page_content='AIR ID\nVendor Name\nApplication Status\nIntegration Specific\nIntegration Approach\nSP/IDP\nIntermediate screen present to enter UPN?\nAD Group Restriction\nSupported by LVA Team\n(Yes/No)\nRoles\nSupports Separate Stage & Production Environments\nSuperGroup\nYes/No\nGroup name vs Object ID\nStage AD Group/Stage SuperGroup Name\nProduction AD Group/Production SuperGroup Name\nProduction AD Group Owner/s\nAuto Provisioning\nName ID\nIf Name ID is not People Key. What is it & why?\nName ID format\nSupport for Prejoiners\nSupport for Contractors\nSupport for V&A\nMobile SSO\nmyL Asset - Yes/No\nIf Q= Yes, then Course Information\nADFS Relying Parties/Azure SSO Decommossioned\nDecommission RITMs\nVendor Contact\nBusiness Owner Contact\nVendor Server Hosting Location Country\nStage Cert Exp\nProd cert Exp\nStage Active / Inactive', metadata={'source': '/content/Sample2.xlsx'}),
 Document(page_content='10040\nPluralsight\nActive\nMigrated to Azure AD\nSP\nNo\nYes, mu

In [4]:
docs

[Document(page_content="\n\n\nAIR ID\nVendor Name\nApplication Status\nIntegration Specific\nIntegration Approach\nSP/IDP\nIntermediate screen present to enter UPN?\nAD Group Restriction\nSupported by LVA Team\n(Yes/No)\nRoles\nSupports Separate Stage & Production Environments\nSuperGroup\nYes/No\nGroup name vs Object ID\nStage AD Group/Stage SuperGroup Name\nProduction AD Group/Production SuperGroup Name\nProduction AD Group Owner/s\nAuto Provisioning\nName ID\nIf Name ID is not People Key. What is it & why?\nName ID format\nSupport for Prejoiners\nSupport for Contractors\nSupport for V&A\nMobile SSO\nmyL Asset - Yes/No\nIf Q= Yes, then Course Information\nADFS Relying Parties/Azure SSO Decommossioned\nDecommission RITMs\nVendor Contact\nBusiness Owner Contact\nVendor Server Hosting Location Country\nStage Cert Exp\nProd cert Exp\nStage Active / Inactive\n\n\n\n\n10040\nPluralsight\nActive\nMigrated to Azure AD\nSP\nNo\nYes, multiple AD groups inside the SG to put users in specific le

In [8]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)


from langchain.vectorstores import Chroma
persist_directory = '/content/chroma/'

# to remove old database files
# !rm -rf ./content/chroma/

db=Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)


In [18]:
# #fetching relevant data directly from vector db using similarilty search
# Query='What is the NameID for Pluralsight vendor'
# # result=db.similarity_search(Query, k=2)
# result1=db.max_marginal_relevance_search(Query,fetch_k=4,k=3)

# print(result1[0].page_content
# ,result1[1].page_content,
# result1[2].page_content)

# from langchain.chat_models import ChatOpenAI
# llm=ChatOpenAI(model_name='gpt-3.5-turbo',temperature=0,openai_api_key= openai.api_key)

# # Build prompt
# from langchain.prompts import PromptTemplate
# template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible.
# {context}
# Question: {question}
# Helpful Answer:"""
# QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template)

# # Run chain
# from langchain.chains import RetrievalQA

# def run_model(question):
#   qa_chain = RetrievalQA.from_chain_type(llm,
#                                        retriever=db.as_retriever(search_type='mmr',search_kwargs={'k': 3, 'fetch_k':4 }),
#                                        return_source_documents=True,
#                                        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}, verbose =True)
#   result=qa_chain({"query":question})
#   return result

In [20]:
 run_model('What is the NameID for Pluralsight vendor')



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'What is the NameID for Pluralsight vendor',
 'result': 'The NameID for the Pluralsight vendor is unspecified.',
 'source_documents': [Document(page_content='10040\nPluralsight\nActive\nMigrated to Azure AD\nSP\nNo\nYes, multiple AD groups inside the SG to put users in specific learning paths.\nYes\nUser, Team Manager, Admin\nYes\nYes\nObject IDs are being passed under groups claim in SAML.\nPluralsight_SSO_SG\nPluralsight_SSO_SG\ncorina.dobos\\noana.florica\\ndijosh.thannikkattil\nYes\nPeopleKey\n\nunspecified\nNo\nNo\nNo', metadata={'category': 'Table', 'file_directory': '/content', 'filename': 'Sample2.xlsx', 'filetype': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'last_modified': '2023-09-19T11:12:44', 'page_name': 'Sheet1', 'page_number': 1, 'source': '/content/Sample2.xlsx', 'text_as_html': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>AIR ID</th>\n      <th>Vendor Name</th>\n      <th>Applicati

In [22]:
# including memory to store past conversation
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
from langchain.chains import ConversationalRetrievalChain

def run_memory_chain(question):
  qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=db.as_retriever(search_type='similarity',search_kwargs={'k': 3, 'fetch_k':4 }),
    chain_type="stuff",
    memory=memory,
    verbose=True)

  result =qa({"question":question})
  return result



In [23]:
run_memory_chain('What is the NameID for Pluralsight vendor?')



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
10040
Pluralsight
Active
Migrated to Azure AD
SP
No
Yes, multiple AD groups inside the SG to put users in specific learning paths.
Yes
User, Team Manager, Admin
Yes
Yes
Object IDs are being passed under groups claim in SAML.
Pluralsight_SSO_SG
Pluralsight_SSO_SG
corina.dobos\noana.florica\ndijosh.thannikkattil
Yes
PeopleKey

unspecified
No
No
No

Yes
PeopleKey

unspecified
No
No
No
No

myL - 360 Multi Rater Assessment - https://mylearning.accenture.com/mylearningui/learner/coursedetail/1822850


ddavis@tracom.com
francine.tatari@accenture.com
AWS Ohio USA
7/5/2024, 11:56:35 AM
10/1/2024, 8:08:12 PM


Active


163354
Google
Active
Azure AD
IDP
No
No
No
User
Yes
No




No, User needs to be added on the 

{'question': 'What is the NameID for Pluralsight vendor?',
 'chat_history': [HumanMessage(content='What is the NameID for Pluralsight vendor?', additional_kwargs={}, example=False),
  AIMessage(content='The provided context does not mention the specific NameID for the Pluralsight vendor.', additional_kwargs={}, example=False)],
 'answer': 'The provided context does not mention the specific NameID for the Pluralsight vendor.'}